<a href="https://colab.research.google.com/github/mohammedterry/NLP_for_ML/blob/master/Sentence_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
example_sentences = [
    # Smartphones
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",

    # Weather
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",

    # Food and health
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",

    # Asking about age
    "How old are you?",
    "what is your age?",
    
    # Reversed Words
    "Cats chase dogs",
    "Dogs chase cats",
]

In [0]:
def most_similar(vector, other_vectors):
  from scipy.spatial.distance import cosine

  scores = [cosine(vector, vec) for vec in other_vectors]
  ranked_candidates_idx = sorted([(score,idx) for idx,score in enumerate(scores)])
  ranked_candidates = [example_sentences[idx] for _,idx in ranked_candidates_idx[:3]]
  return ranked_candidates

# Char2Vec

* Idea:  Convert the sentence (string of characters) into a fixed-size vector (150 dimensions) according to the sequence of characters in its string (takes character order into account).
* Pros:  Relatively Simple
* Cons:  Sentences spellt similarly but with different meanings end up having similar vectors (e.g. "I like my phone" & "I don't like my phone")

In [0]:
!pip3 install chars2vec
import chars2vec
c2v_model = chars2vec.load_model('eng_150')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def char2vec(sentences):
  return c2v_model.vectorize_words(sentences)

In [0]:
c2v_vectors = char2vec(example_sentences)

In [0]:
most_similar(char2vec(["i like my cell phone"]), c2v_vectors)

['My phone is not good.', 'Global warming is real', 'I like my phone']

# Spacy
* Idea: Use spacy's inbuilt method for forming sentence vectors
* Pros: Simple


In [0]:
import spacy
!python3 -m spacy download en_core_web_lg
sp = spacy.load('en_core_web_lg') 

    100% |████████████████████████████████| 852.3MB 53.7MB/s 
  Running setup.py install for en-core-web-lg ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [0]:
spacy_sentences = [sp(sentence) for sentence in example_sentences]

scores = [sp("i like my cell phone").similarity(sp_sent) for sp_sent in spacy_sentences]
ranked_candidates_idx = sorted([(score,idx) for idx,score in enumerate(scores)], reverse = True)
ranked_candidates = [example_sentences[idx] for _,idx in ranked_candidates_idx[:3]]
ranked_candidates

['I like my phone', 'My phone is not good.', 'Your cellphone looks great.']

# Universal Encoder
* Idea: Sentence in -> Fixed-size Vector out (520 dimensions)
* Pros: Regardless of the number of words in the sentence, a single vector is given back
* Cons:  Compute Intensive

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

tf.logging.set_verbosity(tf.logging.ERROR)

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

In [0]:
def universal_sentence(sentences):
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    vectors = session.run(embed(sentences))
    return vectors

In [0]:
uni_vectors = universal_sentence(example_sentences)

In [0]:
most_similar(universal_sentence(["i like my cell phone"]), uni_vectors)

['I like my phone', 'My phone is not good.', 'Your cellphone looks great.']

# BERT as a Sentence Vector

For classification tasks, the first vector (corresponding to [CLS]) is used as as the "sentence vector". Note that this only makes sense because the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]

# Average of Word Vectors

In [0]:
import numpy as np

def mean_vectors(vectors, weights = None):
  return np.average(vectors, axis=0, weights = weights)

## BERT

In [0]:
!pip3 install pytorch_pretrained_bert

from pytorch_pretrained_bert import BertTokenizer
tokeniser = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case = True)

from pytorch_pretrained_bert import BertModel
bert = BertModel.from_pretrained('bert-large-uncased')

import torch

    100% |████████████████████████████████| 122kB 3.7MB/s 
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


100%|██████████| 1248501532/1248501532 [00:19<00:00, 62878262.74B/s]


In [0]:
def convert_to_bert(sentence, model):
  tokenised_sentence = tokeniser.tokenize(sentence)
  tokens_tensor = torch.tensor([tokeniser.convert_tokens_to_ids(tokenised_sentence)])
  segments_tensor = torch.tensor([[0 for _ in range(len(tokenised_sentence))]])
  encoded_layers, _ = model(tokens_tensor, segments_tensor)
  bert_vectors = encoded_layers[0][-1]
  return bert_vectors.detach().numpy()

In [0]:
mean_bert_vectors = [mean_vectors(convert_to_bert(sentence, bert)) for sentence in example_sentences]

In [0]:
most_similar(mean_vectors(convert_to_bert("i like my cell phone", bert)), mean_bert_vectors)

['I like my phone', 'My phone is not good.', 'Your cellphone looks great.']

## ELMO

In [0]:
!pip3 install allennlp
from allennlp.commands.elmo import ElmoEmbedder
elmo = ElmoEmbedder()

    100% |████████████████████████████████| 5.6MB 1.1MB/s 
    100% |████████████████████████████████| 51kB 18.7MB/s 
    100% |████████████████████████████████| 51kB 18.9MB/s 
    100% |████████████████████████████████| 552kB 24.8MB/s 
    100% |████████████████████████████████| 1.5MB 15.6MB/s 
    100% |████████████████████████████████| 12.6MB 2.5MB/s 
    100% |████████████████████████████████| 245kB 23.3MB/s 
    100% |████████████████████████████████| 133kB 32.5MB/s 
    100% |████████████████████████████████| 512kB 20.1MB/s 
    100% |████████████████████████████████| 4.5MB 8.3MB/s 
    100% |████████████████████████████████| 61kB 18.3MB/s 
    100% |████████████████████████████████| 2.3MB 12.5MB/s 
    100% |████████████████████████████████| 143kB 32.2MB/s 
    100% |████████████████████████████████| 51kB 19.3MB/s 
    100% |████████████████████████████████| 51kB 18.5MB/s 
    100% |████████████████████████████████| 102kB 26.9MB/s 
    100% |████████████████████████████████| 204

100%|██████████| 374434792/374434792 [00:14<00:00, 26229429.70B/s]


In [0]:
def convert_to_elmo(sentence, model):
  return elmo.embed_sentence(sentence.split())[-1]

In [0]:
mean_elmo_vectors = [mean_vectors(convert_to_elmo(sentence, elmo)) for sentence in example_sentences]

In [0]:
most_similar(mean_vectors(convert_to_elmo("i like my cell phone", elmo)), mean_elmo_vectors)

['I like my phone', 'Your cellphone looks great.', 'My phone is not good.']

## FastText & GloVe

In [0]:
!pip3 install mxnet
!pip3 install gluonnlp

import gluonnlp
fasttext = gluonnlp.embedding.create('fasttext', source='wiki.simple')
glove = gluonnlp.embedding.create('glove', source='glove.6B.300d') 

Embedding file glove.6B.300d.npz is not found. Downloading from Gluon Repository. This may take some time.


In [0]:
def convert_to_ft_glove(sentence,model):
  return [model[word].asnumpy() for word in sentence.split()]

In [0]:
mean_fasttext_vectors = [mean_vectors(convert_to_ft_glove(sentence, fasttext)) for sentence in sentences]
mean_glove_vectors = [mean_vectors(convert_to_ft_glove(sentence, glove)) for sentence in sentences]

In [0]:
most_similar(mean_vectors(convert_to_ft_glove("i like my cell phone", fasttext)), mean_fasttext_vectors)

['I like my phone', 'My phone is not good.', 'Your cellphone looks great.']

In [0]:
most_similar(mean_vectors(convert_to_ft_glove("i like my cell phone", glove)), mean_glove_vectors)

['I like my phone', 'My phone is not good.', 'what is your age?']

## Number Crunch

# (Tf-Idf) Weighted Average of Word Vectors

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
    
tfidf = TfidfVectorizer()
importance = tfidf.fit_transform(example_sentences) 
feature_names = tfidf.get_feature_names()
tfs = {feature_names[col] : importance[0, col]  for col in importance.nonzero()[1]}

def tfidf_weights(tokens, weights):
  ws = [0.0001 for _ in range(len(tokens))]
  for i,token in enumerate(tokens):
    if token in weights:
      w = weights[token]
      if w > 0:
        ws[i] = w
  return ws

In [0]:
test_sentence = "i like my cell phone"
example_tokens = test_sentence.split()
example_weights = tfidf_weights(example_tokens, tfs)
print(example_weights)
' '.join([example_token.upper()  if tfidf_weight > .5 else example_token for example_token,tfidf_weight in zip(example_tokens,example_weights)])

[0.0001, 0.6340628633768814, 0.5467925956367323, 0.0001, 0.5467925956367323]


'i LIKE MY cell PHONE'

In [0]:
mean_bert_vectors = [mean_vectors(convert_to_bert(sentence, bert), tfidf_weights(tokeniser.tokenize(sentence), tfs)) for sentence in example_sentences]
mean_elmo_vectors = [mean_vectors(convert_to_elmo(sentence, elmo), tfidf_weights(sentence.split(), tfs)) for sentence in example_sentences]
mean_fasttext_vectors = [mean_vectors(convert_to_ft_glove(sentence, fasttext), tfidf_weights(sentence.split(), tfs)) for sentence in sentences]
mean_glove_vectors = [mean_vectors(convert_to_ft_glove(sentence, glove), tfidf_weights(sentence.split(), tfs)) for sentence in sentences]

In [0]:
most_similar(mean_vectors(convert_to_bert(test_sentence, bert), tfidf_weights(tokeniser.tokenize(test_sentence), tfs)), mean_bert_vectors)

['I like my phone', 'My phone is not good.', 'Your cellphone looks great.']

In [0]:
most_similar(mean_vectors(convert_to_elmo(test_sentence, elmo), example_weights), mean_elmo_vectors)

['I like my phone', 'Your cellphone looks great.', 'what is your age?']

In [0]:
most_similar(mean_vectors(convert_to_ft_glove(test_sentence, fasttext),example_weights), mean_fasttext_vectors)

['I like my phone', 'My phone is not good.', 'Your cellphone looks great.']

In [0]:
most_similar(mean_vectors(convert_to_ft_glove(test_sentence, glove), example_weights), mean_glove_vectors)

['I like my phone', 'what is your age?', 'My phone is not good.']

# CNN combined Word Vectors

# Random Projection Word Vectors

# Stacked Autoencoders

# Triplets of Sentence Vectors (Semantic Net Vectors)
Allen NLP's Open Information Extraction to identify triplet.  Then embed

# Benchmarking

https://github.com/facebookresearch/SentEval